<a href="https://colab.research.google.com/github/changyuhsin1999/Fine-tuned-llama-MedQA/blob/main/project_1_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install libraries

In [1]:
!pip install transformers datasets peft accelerate bitsandbytes trl safetensors torch --no-cache

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 215.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 268.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 282.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 345.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 322.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 178.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 225.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 359.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 240.1 MB/s eta 0:00:00
  

## Load dataset and do train split

In [2]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("medalpaca/medical_meadow_medqa", split="train")

print(f"Dataset Size: {len(dataset)}")
print(dataset[randrange(len(dataset))])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Dataset Size: 10178
{'output': 'E: Overproliferation of plasma cells\n"', 'input': 'Q:A 70-year-old man is brought to the emergency department by his wife because of lethargy, confusion, and nausea for the past 2 days. He has previously been healthy and has no past medical history. His only medications are a daily multivitamin and acetaminophen, which he takes daily for hip pain. Vital signs are within normal limits. He is disoriented to place and time but recognizes his wife. The remainder of his physical examination shows no abnormalities. Laboratory studies show a hemoglobin concentration of 9.1 g/dL, a serum calcium concentration of 14.7 mg/dL, and a serum creatinine of 2.2 mg/dL (previously 0.9 mg/dL). Which of the following is the most likely underlying mechanism of this patient\'s condition?? \n{\'A\': \'Excessive consumption of calcium\', \'B\': \'Ectopic PTHrP release\', \'C\': \'Increased serum levels of 1,25-hydroxyvitamin D\', \'D\': \'Excess PTH secretion from parathyroid 

## Prompting

In [31]:
def format_prompt(sample):
    return f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
please give the response in a simple format of A: answer and do not give the explanation.
### Instruction:
{sample["instruction"]}

### Input:
{sample["input"]}

### Response:
{sample["output"]}
"""

In [32]:
from random import randrange

print(format_prompt(dataset[randrange(len(dataset))]))


Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
please give the response in a simple format of A: answer and do not give the explanation.
### Instruction:
Please answer with one of the option in the bracket

### Input:
Q:A 24-year-old man presents to his primary care provider with complaints of 2 days of profuse diarrhea. He states that his stool started to turn watery and lighter in color beginning yesterday, and he has not noticed any fevers. His diarrhea episodes have become more frequent and white-colored over the past day. He has also noticed dry mouth symptoms and darker urine today. He is otherwise healthy but recently returned from a trip with friends to South Asia. None of his friends have reported any symptoms. On exam, his temperature is 98.6°F (37.0°C), blood pressure is 110/68 mmHg, pulse is 80/min, respirations are 14/min. The patient has normal skin turgor, but 

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Hugging Face model name
model_name = "meta-llama/Llama-2-7b-chat-hf"
use_flash_attention = False

# BitsAndBytesConfig int-4 config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    use_cache=False,
    use_flash_attention_2=use_flash_attention,
    device_map="auto",
    torch_dtype=torch.float16
)

model.config.pretraining_tp = 1

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [39]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("medalpaca/medical_meadow_medqa", split="train")
sample = dataset[randrange(len(dataset))]

prompt = f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
please give the response in a format of A: answer

### Instruction:
{sample["instruction"]}

### Input:
{sample["input"]}

### Response:

"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = model.generate(input_ids=input_ids, max_new_tokens=512, do_sample=True, top_p=0.6,temperature=0.9)

print(f"Instruction:\n{sample['instruction']}\n")
print(f"Input:\n{sample['input']}\n")
print(f"Generated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}\n")
print(f"Ground Truth:\n{sample['output']}")

Instruction:
Please answer with one of the option in the bracket

Input:
Q:A 4-year-old boy presents to the emergency department after his parents found him drinking blue liquid out of an unlabeled bottle in the garage. They have no idea what was in the bottle and are concerned for his health. They have brought the bottle with them to the emergency department. The child's past medical history is not remarkable, and he is currently not taking any medications. The patient's vitals are within normal limits for his age. Physical exam reveals a crying child who is drooling. A radiograph is performed, and the child's vitals are closely monitored. It is determined that the blue liquid is a strong alkali. Which of the following is the best next step in management?? 
{'A': 'Administration of a diluent', 'B': 'Administration of a weak acid', 'C': 'Charcoal', 'D': 'Gastrografin swallow and endoscopy', 'E': 'Ipecac'},

Generated Response:
D: Gastrografin swallow and endoscopy


Ground Truth:
D: Ga

In [44]:
# Initialize counters
correct_predictions = 0
total_predictions = 0

# Define the number of samples to evaluate
num_samples = 100  # Adjust this number based on your needs and computational resources

# Loop through the dataset
for _ in range(num_samples):
    # Randomly select a sample from the dataset
    sample = dataset[randrange(len(dataset))]

    # Construct the prompt
    prompt = f"""
    Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    please give the response in a simple format of A: answer and do not give the explanation.

    ### Instruction:
    {sample["instruction"]}

    ### Input:
    {sample["input"]}

    ### Response:

    """

    # Generate the response
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    outputs = model.generate(input_ids=input_ids, max_new_tokens=512, do_sample=True, top_p=0.6, temperature=0.9)
    generated_response = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):].strip()

    # Get the ground truth
    ground_truth = sample["output"]

    # Check if the responses match
    if generated_response == ground_truth:
        correct_predictions += 1
    total_predictions += 1

# Calculate and print the accuracy
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 18.00%


In [10]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

# LoRA config based on QLoRA paper
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.1,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
)
# Prepare model for training
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

In [11]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="finetuned-llama-7b-chat-hf-with-medQA",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="paged_adamw_32bit",
    logging_steps=10,
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    disable_tqdm=False
)

In [12]:
from trl import SFTTrainer

max_seq_length = 1024 # max sequence length for model and packing of the dataset

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_prompt,
    args=args,
)

Generating train split: 0 examples [00:00, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:
# Train
trainer.train()

# Save model
trainer.save_model()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,1.418800
20,1.139700
30,1.000300
40,0.942800
50,0.928100
60,0.910600
70,0.905600
80,0.886300
90,0.904200
100,0.899800


Step,Training Loss
10,1.418800
20,1.139700
30,1.000300
40,0.942800
50,0.928100
60,0.910600
70,0.905600
80,0.886300
90,0.904200
100,0.899800


In [14]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

# Load finetuned LLM model and tokenizer
model_trained = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(args.output_dir)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
from datasets import load_dataset
from random import randrange

# Load dataset from the hub
dataset = load_dataset("medalpaca/medical_meadow_medqa", split="train")
sample = dataset[randrange(len(dataset))]

prompt = f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
please give the response in a format of A: answer

### Instruction:
{sample["instruction"]}

### Input:
{sample["input"]}

### Response:

"""

input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
outputs = model_trained.generate(input_ids=input_ids, max_new_tokens=512, do_sample=True, top_p=0.6,temperature=0.9)

print(f"Instruction:\n{sample['instruction']}\n")
print(f"Input:\n{sample['input']}\n")
print(f"Generated Response:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}\n")
print(f"Ground Truth:\n{sample['output']}")

Instruction:
Please answer with one of the option in the bracket

Input:
Q:A 27 year-old-male presents to the Emergency Room as a code trauma after being shot in the neck. En route, the patient’s blood pressure is 127/73 mmHg, pulse is 91/min, respirations are 14/min, and oxygen saturation is 100% on room air with GCS of 15. On physical exam, the patient is in no acute distress; however, there is an obvious entry point with oozing blood near the left lateral neck above the cricoid cartilage with a small hematoma that is non-pulsatile and stable since arrival. The rest of the physical exam is unremarkable. Rapid hemoglobin returns back at 14.1 g/dL. After initial resuscitation, what is the next best step in management?? 
{'A': 'MRI', 'B': 'Plain radiography films', 'C': 'Conventional angiography', 'D': 'CT angiography', 'E': 'Bedside neck exploration'},

Generated Response:
D: CT angiography


Ground Truth:
D: CT angiography


In [46]:

# Initialize counters
correct_predictions = 0
total_predictions = 0

# Define the number of samples to evaluate
num_samples = 100  # Adjust this number based on your needs and computational resources

# Loop through the dataset
for _ in range(num_samples):
    # Randomly select a sample from the dataset
    sample = dataset[randrange(len(dataset))]

    # Construct the prompt
    prompt = f"""
    Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
    please give the response in a simple format of A: answer and do not give any explanation. please do not add special characters or extra spaces in the response.

    ### Instruction:
    {sample["instruction"]}

    ### Input:
    {sample["input"]}

    ### Response:

    """

    # Generate the response
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    outputs = model_trained.generate(input_ids=input_ids, max_new_tokens=512, do_sample=True, top_p=0.6, temperature=0.9)
    generated_response = tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]
    normalized_generated_response = generated_response.strip().lower()
    normalized_ground_truth = sample['output'].strip().lower()

    # Debugging print statements
    #print(f"Normalized Generated Response: {normalized_generated_response}")
    #print(f"Normalized Ground Truth: {normalized_ground_truth}")

    # Check if the normalized responses match
    if normalized_generated_response == normalized_ground_truth:
        correct_predictions += 1
    total_predictions += 1

# Calculate and print the accuracy
accuracy = correct_predictions / total_predictions
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 20.00%
